In [21]:
import os

In [22]:
os.chdir("../")

In [23]:
%pwd

'/Users/pradhyumnyadav'

In [24]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from dataclasses import dataclass
from pathlib import Path
from ct_scan.constants import *
import tensorflow as tf
from ct_scan.utils.common import create_directories



@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [25]:
class ConfigurationManager:
    def __init__(self):
        self.config = {
            "artifacts_root": "artifacts",
            "data_ingestion": {
                "root_dir": "artifacts/data_ingestion",
                "source_URL": "https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing",
                "local_data_file": "artifacts/data_ingestion/data.zip",
                "unzip_dir": "artifacts/data_ingestion"
            },
            "prepare_base_model": {
                "root_dir": "artifacts/prepare_base_model",
                "base_model_path": "artifacts/prepare_base_model/base_model.h5",
                "updated_base_model_path": "artifacts/prepare_base_model/base_model_updated.h5"
            },
            "training": {
                "root_dir": "artifacts/training",
                "trained_model_path": "artifacts/training/model.h5"
            }
        }
        self.params = {
            "AUGMENTATION": True,
            "IMAGE_SIZE": [224, 224, 3],
            "BATCH_SIZE": 16,
            "INCLUDE_TOP": False,
            "EPOCHS": 5,
            "CLASSES": 2,
            "WEIGHTS": "imagenet",
            "LEARNING_RATE": 0.01
        }
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config["training"]
        prepare_base_model = self.config["prepare_base_model"]
        params = self.params
        training_data = os.path.join(self.config["data_ingestion"]["unzip_dir"], "Chest-CT-Scan-data")
        os.makedirs(Path(training["root_dir"]), exist_ok=True)


        training_config = TrainingConfig(
            root_dir=Path(training["root_dir"]),
            trained_model_path=Path(training["trained_model_path"]),
            updated_base_model_path=Path(prepare_base_model["updated_base_model_path"]),
            training_data=Path(training_data),
            params_epochs=params["EPOCHS"],
            params_batch_size=params["BATCH_SIZE"],
            params_is_augmentation=params["AUGMENTATION"],
            params_image_size=params["IMAGE_SIZE"]
        )

        return training_config


In [26]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [27]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-23 18:11:06,342: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/5
17/17 [==============================] - 29s 2s/step - loss: 14.5829 - accuracy: 0.5367 - val_loss: 10.5387 - val_accuracy: 0.6094
Epoch 2/5
17/17 [==============================] - 29s 2s/step - loss: 14.5999 - accuracy: 0.5444 - val_loss: 0.1099 - val_accuracy: 0.9844
Epoch 3/5
17/17 [==============================] - 30s 2s/step - loss: 9.4188 - accuracy: 0.6023 - val_loss: 2.2510 - val_accuracy: 0.8750
Epoch 4/5
17/17 [==============================] - 35s 2s/step - loss: 7.3329 - accuracy: 0.6602 - val_loss: 9.5366 - val_accuracy: 0.3906
Epoch 5/5
17/17 [==============================] - 33s 2s/step - loss: 6.7428 - accuracy: 0.6255 - val_loss: 19.3017 - val_accuracy: 

/Users/pradhyumnyadav/Desktop/chest_ctscan_analysis/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
